<a href="https://colab.research.google.com/github/kemaleddin222/ag_parametreleri/blob/master/Bitirme_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm *.{jpg,png,pkl,h5,zip}
!rm -r faces
!wget https://github.com/kemaleddin111/test2/raw/master/fcc.zip
!unzip fcc.zip

In [0]:
import torch.nn as nn
import os
import torch
import torch.nn.functional as F
import time
import torchvision.transforms as transform
import torchvision.datasets as dtst
import torch.utils as utils
import pickle
import cv2
from matplotlib import pyplot
from keras.layers import Flatten, Dense, Input, GlobalAveragePooling2D, \
    GlobalMaxPooling2D, Activation, Conv2D, MaxPooling2D, BatchNormalization, \
    AveragePooling2D
from keras.utils.data_utils import get_file
from keras.models import Model
from keras import layers
from keras.preprocessing import image
import numpy as np
from scipy.spatial.distance import cosine
from  torch.autograd import Variable
import itertools




class uretici_ag(nn.Module):
#   agin genel yapisi
    def __init__(self, d=64):
        super(uretici_ag, self).__init__()
        # U-NET tabanli yapi 
        # Konvolusyonlu Kucultme 
        self.konvolusyon1 = nn.Conv2d(3, d, 4, 2, 1)
        self.konvolusyon2 = nn.Conv2d(d, d * 2, 4, 2, 1)
        self.konvolusyon2_batch = nn.BatchNorm2d(d * 2)
        self.konvolusyon3 = nn.Conv2d(d * 2, d * 4, 4, 2, 1)
        self.konvolusyon3_batch = nn.BatchNorm2d(d * 4)
        self.konvolusyon4 = nn.Conv2d(d * 4, d * 8, 4, 2, 1)
        self.konvolusyon4_batch = nn.BatchNorm2d(d * 8)
        self.konvolusyon5 = nn.Conv2d(d * 8, d * 8, 4, 2, 1)
        self.konvolusyon5_batch = nn.BatchNorm2d(d * 8)
        self.konvolusyon6= nn.Conv2d(d * 8, d * 8, 4, 2, 1)
        self.konvolusyon6_batch = nn.BatchNorm2d(d * 8)
        self.konvolusyon7= nn.Conv2d(d * 8, d * 8, 4, 2, 1)
        self.konvolusyon7_batch= nn.BatchNorm2d(d * 8)
        self.konvolusyon8= nn.Conv2d(d * 8, d * 8, 4, 2, 1)
        # self.conv8_bn = nn.BatchNorm2d(d * 8)

        # Ters Konvolusyon (Transpoze Konvolusyonlu Buyultme)
        self.ters_konvolusyon1 = nn.ConvTranspose2d(d * 8, d * 8, 4, 2, 1)
        self.ters_konvolusyon1_batch = nn.BatchNorm2d(d * 8)
        self.ters_konvolusyon2= nn.ConvTranspose2d(d * 8 * 2, d * 8, 4, 2, 1)
        self.ters_konvolusyon2_batch = nn.BatchNorm2d(d * 8)
        self.ters_konvolusyon3= nn.ConvTranspose2d(d * 8 * 2, d * 8, 4, 2, 1)
        self.ters_konvolusyon3_batch= nn.BatchNorm2d(d * 8)
        self.ters_konvolusyon4= nn.ConvTranspose2d(d * 8 * 2, d * 8, 4, 2, 1)
        self.ters_konvolusyon4_batch= nn.BatchNorm2d(d * 8)
        self.ters_konvolusyon5= nn.ConvTranspose2d(d * 8 * 2, d * 4, 4, 2, 1)
        self.ters_konvolusyon5_batch= nn.BatchNorm2d(d * 4)
        self.ters_konvolusyon6= nn.ConvTranspose2d(d * 4 * 2, d * 2, 4, 2, 1)
        self.ters_konvolusyon6_batch= nn.BatchNorm2d(d * 2)
        self.ters_konvolusyon7= nn.ConvTranspose2d(d * 2 * 2, d, 4, 2, 1)
        self.ters_konvolusyon7_batch= nn.BatchNorm2d(d)
        self.ters_konvolusyon8= nn.ConvTranspose2d(d * 2, 3, 4, 2, 1)

    # Kitle (Matrisin ilkin katsayilarinin ayarlanmasi)
    def _katsayi_ayarla_ag_(self, ortalama, standart_sapma):
        for m in self._modules:
            katsayi_ayarlama(self._modules[m], ortalama, standart_sapma)

    # forward method
    def forward(self, girdi):
      # Bu kisimda yayindada belirtildigi gibi U-Net agini kurmaya basliyoruz
        e1 = self.konvolusyon1(girdi)
        e2 = self.konvolusyon2_batch(self.konvolusyon2(F.leaky_relu(e1, 0.2)))
        e3 = self.konvolusyon3_batch(self.konvolusyon3(F.leaky_relu(e2, 0.2)))
        e4 = self.konvolusyon4_batch(self.konvolusyon4(F.leaky_relu(e3, 0.2)))
        e5 = self.konvolusyon5_batch(self.konvolusyon5(F.leaky_relu(e4, 0.2)))
        e6 = self.konvolusyon6_batch(self.konvolusyon6(F.leaky_relu(e5, 0.2)))
        e7 = self.konvolusyon7_batch(self.konvolusyon7(F.leaky_relu(e6, 0.2)))
        e8 = self.konvolusyon8(F.leaky_relu(e7, 0.2))

        
        d1 = F.dropout(self.ters_konvolusyon1_batch(self.ters_konvolusyon1(F.relu(e8))), 0.5, training=True)
        d1 = torch.cat([d1, e7], 1)
        d2 = F.dropout(self.ters_konvolusyon2_batch(self.ters_konvolusyon2(F.relu(d1))), 0.5, training=True)
        d2 = torch.cat([d2, e6], 1)
        d3 = F.dropout(self.ters_konvolusyon3_batch(self.ters_konvolusyon3(F.relu(d2))), 0.5, training=True)
        d3 = torch.cat([d3, e5], 1)
        d4 = self.ters_konvolusyon4_batch(self.ters_konvolusyon4(F.relu(d3)))
        # d4 = F.dropout(self.deconv4_bn(self.deconv4(F.relu(d3))), 0.5)
        d4 = torch.cat([d4, e4], 1)
        d5 = self.ters_konvolusyon5_batch(self.ters_konvolusyon5(F.relu(d4)))
        d5 = torch.cat([d5, e3], 1)
        d6 = self.ters_konvolusyon6_batch(self.ters_konvolusyon6(F.relu(d5)))
        d6 = torch.cat([d6, e2], 1)
        d7 = self.ters_konvolusyon7_batch(self.ters_konvolusyon7(F.relu(d6)))
        d7 = torch.cat([d7, e1], 1)
        d8 = self.ters_konvolusyon8(F.relu(d7))
        o = F.tanh(d8)

        return o

class ayirt_edici_ag(nn.Module):
    # genel ayirt edici ag yapisi
    def __init__(self, d=64):
        super(ayirt_edici_ag, self).__init__()
        self.konvolusyon1 = nn.Conv2d(6, d, 4, 2, 1)
        self.konvolusyon2 = nn.Conv2d(d, d * 2, 4, 2, 1)
        self.konvolusyon2_batch = nn.BatchNorm2d(d * 2)
        self.konvolusyon3 = nn.Conv2d(d * 2, d * 4, 4, 2, 1)
        self.konvolusyon3_batch = nn.BatchNorm2d(d * 4)
        self.konvolusyon4= nn.Conv2d(d * 4, d * 8, 4, 1, 1)
        self.konvolusyon4_batch = nn.BatchNorm2d(d * 8)
        self.konvolusyon5 = nn.Conv2d(d * 8, 1, 4, 1, 1)

    # agin katsayilarinin ayarlanmasi 
    def _katsayi_ayarla_ag_(self, ortalama, standart_sapma):
        for m in self._modules:
            katsayi_ayarlama(self._modules[m], ortalama, standart_sapma)

    # girdi-cikti methodu (forward-pass)
    def forward(self, girdi, cevap):
        x = torch.cat([girdi, cevap], 1)
        x = F.leaky_relu(self.konvolusyon1(x), 0.2)
        x = F.leaky_relu(self.konvolusyon2_batch(self.konvolusyon2(x)), 0.2)
        x = F.leaky_relu(self.konvolusyon3_batch(self.konvolusyon3(x)), 0.2)
        x = F.leaky_relu(self.konvolusyon4_batch(self.konvolusyon4(x)), 0.2)
        x = F.sigmoid(self.konvolusyon5(x))

        return x

def katsayi_ayarlama(katman, ortalama, standart_sapma):
  # Sadece ve Sadece verilen katman Konvolusyon ise o zaman katsayilari ayarla
  # normal dagilima gore ayarlama yap!
    if isinstance(katman, nn.ConvTranspose2d) or isinstance(katman, nn.Conv2d):
        katman.weight.data.normal_(ortalama, standart_sapma)
        katman.bias.data.zero_()

# fotograflari torchun anlaya bilicegi sekilde duzenleyip tensor formuna salalim
def dataloader_hazirla():
    transformation=transform.Compose([transform.ToTensor(),transform.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])])
    traindata=dtst.ImageFolder(root="./faces/img_dataset/",transform=transformation)
    traindataloader=utils.data.DataLoader(traindata,batch_size=10,shuffle=True,num_workers=4)
    return traindataloader

uretici=uretici_ag()
ayirtedici=ayirt_edici_ag()
uretici._katsayi_ayarla_ag_(ortalama=0.0, standart_sapma=0.02)
ayirtedici._katsayi_ayarla_ag_(ortalama=0.0, standart_sapma=0.02)
uretici.cuda()
ayirtedici.cuda()
uretici.train()
ayirtedici.train()


uretici_optimizer=torch.optim.Adam(uretici.parameters(),lr=0.0002,betas=[0.5,0.999])
ayirtedici_optimizer=torch.optim.Adam(ayirtedici.parameters(),lr=0.0002,betas=[0.5,0.999])
hata1=torch.nn.BCELoss().cuda()
hata2=torch.nn.L1Loss().cuda()


def goster(G, kaydet= 'sonuc_foto.png'):
    pyplot.imshow((G[0].cpu().data.numpy().transpose(1, 2, 0) + 1) / 2)
    pyplot.savefig(kaydet,bbox_inches='tight', pad_inches=0,transparent=True)
    pyplot.show()



dataloader=dataloader_hazirla()
transformation=transform.Compose([transform.Resize((256,256)),transform.ToTensor(),transform.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])])
test=dtst.ImageFolder(root="./faces/arama",transform=transformation)
test=utils.data.DataLoader(test,batch_size=1,shuffle=True,num_workers=4)






In [0]:
##############################################################################################
#                             Egitim Kismi
##############################################################################################

#   Pi2Pix aginin egitimi icin bu bloku calistirmak gerekmektedir
#   Bu blok calistiktan sonra her 10 iterasyonda test icin arama klasorundeki
#   Fotolar uretilmeye baslayacak ve gosterilecektir 
#   Eger hazir katsayilardan egitim yapilmadan test edilmek isteniyorsa
#   Diger bu blogu *calistirmadan diger blogu direk calistirin 
#   Iyi sonuclar gore bilmek icin agi en az 2 saat calistirmak gerekiyor 
#   Ortalama sonuc icin 90 devir yeterlidir
#   Cok daha gercekci sonuclar icin en az 1-2 gun calistirma oneriliyor  

iterasyon_sayac=0

for devir_sayisi in range(300):
    for resim, _ in dataloader:
        # ayirtedici agin gradientlerini sifirla
        ayirtedici.zero_grad()
        # sadece cizilmis resmi al 
        cikti = resim[:, :, :, 0:256]
        girdi = resim[:, :, :, 256:]
        
        girdi, cikti = Variable(girdi.cuda()), Variable(cikti.cuda())

        # Ayirt edici agi egit
        ayirt_edici_cikti = ayirtedici(girdi, cikti).squeeze()
        ayirt_edici_hata1 = hata1(ayirt_edici_cikti, Variable(torch.ones(ayirt_edici_cikti.size()).cuda()))

        uretici_cikti = uretici(girdi)
        ayirt_edici_cikti = ayirtedici(girdi, uretici_cikti).squeeze()
        ayirt_edici_hata2 = hata1(ayirt_edici_cikti, Variable(torch.zeros(ayirt_edici_cikti.size()).cuda()))

        ayirt_edici_toplam_hata = (ayirt_edici_hata1 + ayirt_edici_hata2) * 0.5
        ayirt_edici_toplam_hata.backward()
        ayirtedici_optimizer.step()



        # Uretici agin egitimi
        uretici.zero_grad()
        uretici_cikti = uretici(girdi)
        ayirt_edici_cikti = ayirtedici(girdi, uretici_cikti).squeeze()
        uretici_hata = hata1(ayirt_edici_cikti, Variable(torch.ones(ayirt_edici_cikti.size()).cuda())) + 100 * hata2(uretici_cikti, cikti)
        uretici_hata.backward()
        uretici_optimizer.step()

        iterasyon_sayac+= 1
        if(iterasyon_sayac%10==0):
          img=next(iter(test))
          print("Iterasyon Sayisi : {}".format(iterasyon_sayac))
          goster(uretici(Variable(img[0]).to(device="cuda",dtype=torch.float)))
    print("Ag parametreleri ve katsayilari kaydediliyor")
    print("Devir Sayisi : {} ".format(devir_sayisi))
    torch.save(uretici.state_dict(), './uretici.pkl')
    torch.save(ayirtedici.state_dict(), './ayirt_edici.pkl')

In [0]:
################################################################################
#                     Hazir daha once egitimi yapilmis katsayilari yuklemek
#                     icin bu blok , eger yukardaki blokla egitim yapiliyorsa
#               ******bu blok-un calistirilmasina ihtiyac yoktur
################################################################################

!rm *.pkl
!wget https://github.com/kemaleddin222/ag_parametreleri/raw/master/uretici.pkl
!wget https://github.com/kemaleddin222/ag_parametreleri/raw/master/ayirt_edici.pkl


uretici=uretici_ag()
katsayilar=torch.load("uretici.pkl")
uretici.load_state_dict(katsayilar)
uretici=uretici.cuda()

In [0]:
sayac=0
for foto,counter in test:
  res=uretici(Variable(foto).cuda())
  print(res.shape)
  sayac+=1
  goster(res,kaydet=str(sayac)+".jpg")

In [0]:

def Normal_Blok(girdi_tensor, kernel_boyutu, katmanlar,
                          bias=False):
    katman1, katman2, katman3 = katmanlar

    x = Conv2D(katman1, (1, 1), use_bias=bias,)(
        girdi_tensor)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)

    x = Conv2D(katman2, kernel_boyutu, use_bias=bias,
               padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)

    x = Conv2D(katman3, (1, 1), use_bias=bias)(x)
    x = BatchNormalization(axis=3)(x)

    # Burada katmanlar birlestiriliyor
    x = layers.add([x, girdi_tensor])
    x = Activation('relu')(x)
    return x


def Resnet_Blok(girdi_tensor, kernel_boyutu, katmanlar,
                      stride=(2, 2), bias=False):
    katman1,katman2,katman3=katmanlar
    
    x = Conv2D(katman1, (1, 1), strides=stride, use_bias=bias
               )(girdi_tensor)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)

    x = Conv2D(katman2, kernel_boyutu, padding='same', use_bias=bias,
               )(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)

    x = Conv2D(katman3, (1, 1),  use_bias=bias)(x)
    x = BatchNormalization(axis=3)(x)

    katman_hafizasi = Conv2D(katman3, (1, 1), strides=stride, use_bias=bias,
                      )(girdi_tensor)
    katman_hafizasi = BatchNormalization(axis=3)(
        katman_hafizasi)

    x = layers.add([x, katman_hafizasi])
    x = Activation('relu')(x)
    return x


def RESNET50():
    girdi_boyutu = (224,224,3)

    foto_girdi =Input(tensor=None, shape=girdi_boyutu)

    print("Girdi boyutlari: {}",girdi_boyutu)
    x = Conv2D(
        64, (7,7), use_bias=False, strides=(2, 2), padding='same'
      )(foto_girdi)

    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    print("Ilk blok cikti boyutlari: {}",x.shape)


    x = Resnet_Blok(x, 3, [64, 64, 256],stride=(1, 1))
    print("Ikinci Blok boyutlari: {}",x.shape)
    x = Normal_Blok(x, 3, [64, 64, 256])
    print("Ucuncu Blok boyutlari: {}",x.shape)
    x = Normal_Blok(x, 3, [64, 64, 256])
    print("Dorduncu Blok boyutlari: {}",x.shape)
    x = Resnet_Blok(x, 3, [128, 128, 512])
    print("Besinci Blok boyutlari: {}",x.shape)
    x = Normal_Blok(x, 3, [128, 128, 512])
    print("Altinci Blok boyutlari: {}",x.shape)
    x = Normal_Blok(x, 3, [128, 128, 512])
    print("Yedinci Blok boyutlari: {}",x.shape)
    x = Normal_Blok(x, 3, [128, 128, 512])
    print("Sekizinci Blok boyutlari: {}",x.shape)
####
    x = Resnet_Blok(x, 3, [256, 256, 1024])
    print("Dokuzuncu Blok boyutlari: {}",x.shape)

    x = Normal_Blok(x, 3, [256, 256, 1024])
    print("Onuncu Blok boyutlari: {}",x.shape)
    x = Normal_Blok(x, 3, [256, 256, 1024])
    print("Onbirinci Blok boyutlari: {}",x.shape)
    x = Normal_Blok(x, 3, [256, 256, 1024])
    print("Onikinci Blok boyutlari: {}",x.shape)
    x = Normal_Blok(x, 3, [256, 256, 1024])
    print("Onucuncu Blok boyutlari: {}",x.shape)
    x = Normal_Blok(x, 3, [256, 256, 1024])
    print("Ondorduncu Blok boyutlari: {}",x.shape)
####
    x = Resnet_Blok(x, 3, [512, 512, 2048])
    print("Onbesinci Blok boyutlari: {}",x.shape)
    x = Normal_Blok(x, 3, [512, 512, 2048])
    print("Onaltinci Blok boyutlari: {}",x.shape)
    x = Normal_Blok(x, 3, [512, 512, 2048])
    print("Onyedinci Blok boyutlari: {}",x.shape)
    x = AveragePooling2D((7, 7))(x)
    x = GlobalMaxPooling2D()(x)
    print("Son Blok boyutlari: {}",x.shape)

    model = Model(foto_girdi, x, name='vggface_resnet50')
    weights_path = get_file('rcmalli_vggface_tf_notop_resnet50.h5',
                                    'https://github.com/kemaleddin111/Ag_parametreleri/raw/master/resnet_parametreler.h5',
                                    cache_subdir='./')
            
    model.load_weights(weights_path)
    return model

In [0]:
################################################################################
#                    Aranan kisinin resmini buarada seciyoruz
Aranan_ag_ciktisi="4.jpg"

################################################################################

In [0]:
model=RESNET50()

import cv2
aranan_kisi = image.load_img(Aranan_ag_ciktisi,(224, 224))

x = image.img_to_array(aranan_kisi)
x=x[16:241,16:241]
pyplot.imshow(x/255)
pyplot.show()
x=cv2.resize(x,(190,190))
x=cv2.resize(x,(224,224))

x = np.expand_dims(x, axis=0)
aranan_kisi=model.predict(x)


folder="./faces/polis_database/"
dirs=os.listdir(folder)

final_skorlari={}



for dir in dirs:
  imgs=os.listdir(folder+dir)
  scores=[]
  for img in imgs:
    img = image.load_img(folder+dir+"/"+img)
    x = image.img_to_array(img)
    x=cv2.resize(x,(190,190))
    x=cv2.resize(x,(224,224))
    x = np.expand_dims(x, axis=0)
    res1=model.predict(x)
    scores.append(1-cosine(res1[0],aranan_kisi[0]))
  final_skorlari[max(scores)]=dir

maxs=[x for x in sorted(final_skorlari.keys(),reverse=True)]
final=[]
for x in range(3):
  imgs=os.listdir("./faces/polis_database/"+final_skorlari[maxs[x]])
  final.append(pyplot.imread("./faces/polis_database/"+final_skorlari[maxs[x]]+"/"+imgs[0]))



print("En benzer fotolar ve benzerlik skorlari")
pyplot.subplot(131)
pyplot.imshow(final[0])
print("Benzerlik skoru : {:.3f} isim : {}".format(maxs[0]*100,final_skorlari[maxs[0]]))
pyplot.subplot(132)
pyplot.imshow(final[1])
print("Benzerlik skoru : {:.3f} isim : {}".format(maxs[1]*100,final_skorlari[maxs[1]]))
pyplot.subplot(133)
pyplot.imshow(final[2])
print("Benzerlik skoru : {:.3f} isim : {}".format(maxs[2]*100,final_skorlari[maxs[2]]))
pyplot.show()





In [0]:
final_skorlari